In [17]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [2]:
N = 100
L = 10.0
T = 1000
dt = 0.05

plt.rcParams['animation.embed_limit'] = 1000

radius = np.full((N, 1), 0.1)
pos = np.random.uniform(0, L, size=(N, 3))
vel = np.random.uniform(-2, 2, size=(N, 3))
mass = np.ones((N, 1))

In [3]:
def walls_reflect(pos, vel, L, x, y,z):


    vel[x, 0] *= -1
    pos[x, 0] = np.where(pos[x, 0] < 0, 0.0, np.where(pos[x, 0] > L, L, pos[x, 0]))


    vel[y, 1] *= -1
    pos[y, 1] = np.where(pos[y, 1] < 0, 0.0, np.where(pos[y, 1] > L, L, pos[y, 1]))

    vel[z, 2] *= -1
    pos[z, 2] = np.where(pos[z, 2] < 0, 0.0, np.where(pos[z, 2] > L, L, pos[z, 2]))

In [10]:
def step_once(pos,vel,dt,L):

    pos = pos + vel * dt

    x = (pos[:, 0] < 0) | (pos[:, 0] > L)
    y = (pos[:, 1] < 0) | (pos[:, 1] > L)
    z = (pos[:, 2] < 0) | (pos[:, 2] > L)
    walls_reflect(pos, vel, L, x, y,z)

    return pos

In [5]:
def kinetic_energy(mass, vel):
    v_2 = np.sum(vel**2, axis=1)
    return 0.5 * np.sum(mass.flatten() * v_2)

In [6]:
def detect_collisions(pos, radius):
    d = pos[None, :, :] - pos[:, None, :]
    r = np.linalg.norm(d, axis=2)
    R_sum = radius + radius.T
    colliding = (r < R_sum) & (r > 1e-12)
    pairs = np.array(np.where(colliding)).T
    return pairs[pairs[:, 0] < pairs[:, 1]]

In [7]:
def merge_particles(pos, vel, mass, radius, i, j):

    radius[i, 0] = np.sqrt(radius[i, 0]**2 + radius[j, 0]**2)
    total_mass = mass[i, 0] + mass[j, 0]
    vel[i] = (mass[i, 0]*vel[i] + mass[j, 0]*vel[j]) / total_mass
    mass[i, 0] = total_mass
    pos[i] = (pos[i] + pos[j]) / 2
    pos = np.delete(pos, j, axis=0)
    vel = np.delete(vel, j, axis=0)
    mass = np.delete(mass, j, axis=0)
    radius = np.delete(radius, j, axis=0)
    return pos, vel, mass, radius

In [ ]:
plt.close('all')
fig = plt.figure(figsize=(10,5), dpi=150)
ax = fig.add_subplot(121, projection='3d')
ax_energy = fig.add_subplot(122)
plt.subplots_adjust(wspace=0.6)

ax.set_xlim(0, L)
ax.set_ylim(0, L)
ax.set_zlim(0, L)
ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Z')

ax_energy.set_xlabel("Time (s)")
ax_energy.set_ylabel("Energy (J)")
ax_energy.set_title("Energy vs Time")

ke_line, = ax_energy.plot([], [], color='tab:red', label="KE")
ax_energy.legend(loc="upper left", bbox_to_anchor=(1.02,1))
ke_times, ke_values = [], []


scatter = ax.scatter(pos[:,0], pos[:,1], pos[:,2], s=(radius.flatten()*500)**2, c='tab:blue', alpha=0.8)


def init():
    scatter._offsets3d = (pos[:,0], pos[:,1], pos[:,2])
    ke_line.set_data([], [])
    ke_times.clear()
    ke_values.clear()
    return scatter, ke_line

def animate(frame):
    global pos, vel, mass, radius


    pos = step_once(pos, vel, dt,L)


    while True:
        pairs = detect_collisions(pos, radius)
        if len(pairs) == 0:
            break
        i, j = pairs[0]
        pos, vel, mass, radius = merge_particles(pos, vel, mass, radius, i, j)


    scatter._offsets3d = (pos[:,0], pos[:,1], pos[:,2])
    scatter.set_sizes((radius.flatten()*70)**2)

    t = (frame+1) * dt
    KE = kinetic_energy(mass, vel)
    ke_times.append(t)
    ke_values.append(KE)
    ke_line.set_data(ke_times, ke_values)

    ax_energy.set_xlim(0, t+dt)
    ax_energy.set_ylim(0, max(ke_values)*1.2)

    return scatter, ke_line

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=10, blit=True)

# Save as HTML
with open("Merging_3D.html", "w") as f:
    f.write(anim.to_jshtml())
# HTML(anim.to_jshtml())